## WIDER Dataset - DataLoader

In [1]:
# !pip install --upgrade torch --quiet

In [2]:
import json
import io
import boto3
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import Subset
import numpy as np
from PIL import Image
from torchvision.transforms import transforms, RandAugment
import torch
import tarfile
import os
import getpass
import s3fs
import json
from urllib.parse import urlparse
from data.data_loader import load_cifar10, load_cifar100, load_imagenet, load_prof, load_wider


### Data Download - Do not need to repeat this step

In [2]:
# Option 1
# # !aws s3 cp s3://210bucket/wider_attribute_image.tgz ./wider_attribute_image.tgz
# Option 2
# s3 = boto3.client('s3')
# s3.download_file('210bucket', 'wider_attribute_image.tgz', 'wider_attribute_image.tgz')

In [3]:
# # Extract the .tgz file
# with tarfile.open('wider_attribute_image.tgz', 'r:gz') as tar:
#     tar.extractall(path="local_wider")

In [4]:
# # Upload the image files to s3 Bukcet
# s3 = boto3.client('s3')

# def upload_dir_to_s3(bucket_name, s3_folder, local_dir):
#     for root, dirs, files in os.walk(local_dir):
#         for file in files:
#             local_path = os.path.join(root, file)
#             relative_path = os.path.relpath(local_path, local_dir)
#             s3_path = os.path.join(s3_folder, relative_path)
            
#             s3.upload_file(local_path, bucket_name, s3_path)

# upload_dir_to_s3('210bucket', 'WIDER/', 'WIDER/Image/')


### DataLoader

In [5]:
######################### now available using "from data.data_loader import" #########################

# class DataSet(Dataset):
#     def __init__(self, ann_files, augs, img_size, dataset):
#         self.dataset = dataset
#         self.ann_files = ann_files
#         self.augment = self.augs_function(augs, img_size)
#         self.transform = transforms.Compose(
#             [
#                 transforms.ToTensor(),
#                 transforms.Normalize(mean=[0, 0, 0], std=[1, 1, 1])
#             ]
#         )
#         self.anns = []
#         self.s3_client = boto3.client('s3') 
#         self.load_anns()
#         print(self.augment)

#         if self.dataset == "wider":
#             self.transform = transforms.Compose(
#                 [
#                     transforms.ToTensor(),
#                     transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
#                 ]
#             )

#     def extract_label_from_filename(self, file_name):
#         # Split the path and extract the part with '--'
#         parts = file_name.split('/')
#         for part in parts:
#             if '--' in part:
#                 # Extract the numeric part before '--'
#                 label = part.split('--')[0]
#                 if label.isdigit():
#                     return int(label)  # Return as an integer
#         raise ValueError(f"Label not found in file name: {file_name}")

        

#     def augs_function(self, augs, img_size):            
#         t = []
#         if 'randomflip' in augs:
#             t.append(transforms.RandomHorizontalFlip())
#         if 'ColorJitter' in augs:
#             t.append(transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0))
#         if 'resizedcrop' in augs:
#             t.append(transforms.RandomResizedCrop(img_size, scale=(0.7, 1.0)))
#         if 'RandAugment' in augs: # need to review RandAugment()
#             t.append(RandAugment())
#             # t.append(transforms.RandomApply([
#             #     transforms.RandomRotation(degrees=10),
#             #     transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
#             #     transforms.RandomPerspective(distortion_scale=0.05)
#             # ], p=0.5))

#         t.append(transforms.Resize((img_size, img_size)))
    
#         return transforms.Compose(t)

#     def load_anns(self):
#         s3_client = boto3.client('s3')
#         self.anns = []
#         for ann_file in self.ann_files:
#             bucket, key = self.parse_s3_path(ann_file)
#             response = s3_client.get_object(Bucket=bucket, Key=key)
#             json_data = json.loads(response['Body'].read())
#             for image in json_data['images']:
#                 file_name = image['file_name']
#                 label = self.extract_label_from_filename(file_name)  # Use the new method
#                 for target in image['targets']:
#                     ann = {
#                         'img_path': f's3://210bucket/WIDER/{file_name}',
#                         'bbox': target['bbox'],
#                         'label': label,  
#                         'target': target['attribute']
#                     }
#                     self.anns.append(ann)
#         print(f"Loaded annotations: {len(self.anns)}")


#     def __len__(self):
#         return len(self.anns)

#     def __getitem__(self, idx):
#         idx = idx % len(self)
#         ann = self.anns[idx]
    
#         if not isinstance(ann, dict) or "img_path" not in ann:
#             raise ValueError(f"Annotation at index {idx} is not a dictionary with an 'img_path' key: {ann}")
    
#         bucket, key = self.parse_s3_path(ann["img_path"])
#         try:
#             response = self.s3_client.get_object(Bucket=bucket, Key=key)
#             img = Image.open(io.BytesIO(response['Body'].read())).convert("RGB")
#         except self.s3_client.exceptions.NoSuchKey:
#             print(f"File not found: s3://{bucket}/{key}")
#             img = self.get_placeholder_image()
#             return None 
    
#         x, y, w, h = ann['bbox']
#         img = img.crop((x, y, x + w, y + h))
#         img = self.augment(img)
#         img = self.transform(img)
    
#         label = torch.tensor(ann['label'], dtype=torch.long)    
#         target = torch.tensor(ann['target'], dtype=torch.float32)
    
#         message = {
#             "label": label,  
#             "target": target,  
#             "img": img 
#         }
    
#         return message


#     def get_placeholder_image(self):
#         return Image.new('RGB', (256, 256), color = 'gray')

#     @staticmethod
#     def parse_s3_path(s3_path):
#         if not s3_path.startswith("s3://"):
#             raise ValueError(f"Invalid S3 path: {s3_path}")
#         s3_path = s3_path[5:]
#         bucket, key = s3_path.split('/', 1)
#         return bucket, key

In [6]:
######################### now available using "from data.data_loader import" #########################

# def load_wider(batch_size=64, subset_size=None):

#     train_augs = ['randomflip', 'ColorJitter', 'resizedcrop', 'RandAugment']
#     test_augs = []  
#     img_size = 256 
    
#     train_dataset = DataSet(
#         ann_files=['s3://210bucket/wider_attribute_annotation/wider_attribute_trainval.json'],  
#         augs=train_augs,
#         img_size=img_size,
#         dataset='wider'
#     )
    
#     test_dataset = DataSet(
#         ann_files=['s3://210bucket/wider_attribute_annotation/wider_attribute_test.json'], 
#         augs=test_augs,
#         img_size=img_size,
#         dataset='wider'
#     )
    
#     if subset_size is not None:
#         train_indices = np.random.choice(len(train_dataset), subset_size, replace=False)
#         test_indices = np.random.choice(len(test_dataset), subset_size, replace=False)
        
#         train_subset = Subset(train_dataset, train_indices)
#         test_subset = Subset(test_dataset, test_indices)
        
#         train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
#         test_loader = DataLoader(test_subset, batch_size=batch_size, shuffle=False)
#     else:
#         train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#         test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

#     return train_loader, test_loader


In [3]:
trainloader, testloader  = load_wider(batch_size=64)

Loaded annotations: 28345
Compose(
    RandomHorizontalFlip(p=0.5)
    ColorJitter(brightness=(0.5, 1.5), contrast=(0.5, 1.5), saturation=(0.5, 1.5), hue=None)
    RandomResizedCrop(size=(256, 256), scale=(0.7, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear, antialias=warn)
    RandAugment(num_ops=2, magnitude=9, num_magnitude_bins=31, interpolation=InterpolationMode.NEAREST, fill=None)
    Resize(size=(256, 256), interpolation=bilinear, max_size=None, antialias=warn)
)
Loaded annotations: 29179
Compose(
    Resize(size=(256, 256), interpolation=bilinear, max_size=None, antialias=warn)
)
